Okay! Let's try the gym business. Cliff walking environment is perfect.

In [1]:
import gym
import numpy as np
import ToyEnv
import psrl
import PosteriorSamplerGMM

In [2]:
env = gym.make('CliffWalking-v0')

In [3]:
observation, info = env.reset()
print(observation)
print(info)
np.unravel_index(observation, env.shape)

36
{'prob': 1}


(3, 0)

In [4]:
action = env.action_space.sample()
print(action)
observation, reward, terminated, truncated, info = env.step(3)
print(observation)
print(env.action_space.n)
print(reward)
print(env.P[observation][2])

1
36
4
-1
[(1.0, 36, -1, False)]


/home/srayan/miniconda3/envs/tsmdp/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


# Testing ToyEnv

In [5]:
new_env = ToyEnv.ToyEnv()

In [6]:
init_state = new_env.reset()
print(init_state)

1


In [7]:
next_state, reward = new_env.step(1)

In [8]:
print(reward)
print(next_state)

4.002490364475818
3


# Testing Gibbs sampler

In [9]:
# Generate some data for state-action pair (0, 2)
means = new_env._compute_means(0, 2)
sigma = new_env.true_Sigma
pis = new_env.true_pi
data = np.zeros((100,))
for i in range(100):
    k = np.random.choice(len(pis), p=pis)
    data[i] = np.random.normal(loc=means[k], scale=sigma[k])

In [13]:
print(means)
print(sigma)
print(pis)

[-2  0  2]
[0.1 0.1 0.1]
[0.16666667 0.16666667 0.66666667]


In [11]:
gibbs = PosteriorSamplerGMM.GibbsSamplerGMM(1000, 3)

In [12]:
samples = gibbs.fit(data)

In [13]:
print(samples[-1])

(array([-0.41125015, -2.0563082 ,  2.01356477]), array([0.75246343, 0.3374355 , 0.07134082]), array([0.19521561, 0.1451492 , 0.65963519]))
